**INPAINTING-BASED AUGMENTATION FOR CROPS**

This script performs inpainting-based data augmentation by inserting new crop instances into existing field images using a Stable Diffusion model fine-tuned via DreamBooth.

After training a custom model (model_crops) to learn the visual appearance of a specific crop (e.g., sugar beet), this script uses inpainting to:
- Mask part of an input image (left or right portion).
- Prompt the model to fill in the masked area with new, realistic crops.
- Generate augmented training samples with richer crop diversity and spatial configurations.

This method is only applied to crop augmentation, not to weeds, and is crucial for enriching the dataset with synthetic but coherent variations of crops class.

*Library Imports and Initial Setup*

In [ ]:
import torch
import numpy as np
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline
import cv2
import os
import matplotlib.pyplot as plt

image_dir = "images_to_modify"
prompt = "sks crop"
output_dir = "inpainting_outputs"
os.makedirs(output_dir, exist_ok=True)

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "models/model_crops",
    torch_dtype=torch.float16
).to("cuda")

This function creates a binary mask (also called an overlay) that defines the region of the image to be inpainted.
Depending on the specified width_fraction and direction, it masks a portion of the image on the left or right side, allowing the model to insert new crop instances in that area.

The resulting mask is used as input for the inpainting pipeline.

In [ ]:
def generate_overlay(image, width_fraction=0.3, direction='right'):
    width, height = image.size
    overlay = Image.new("L", (width, height), 0)
    start_x = int(width * (1 - width_fraction)) if direction == 'right' else 0
    end_x = width if direction == 'right' else int(width_fraction * width)

    for x in range(start_x, end_x):
        for y in range(height):
            overlay.putpixel((x, y), 255)

    return overlay

This function runs the inpainting process using the fine-tuned Stable Diffusion pipeline.

Key settings:
- prompt: guides the content to be generated in the masked area (e.g., "sks crop").
- overlay: the binary mask indicating where new content should be generated.
- height and width are both set to 1024 to match the resolution of the original dataset images.
- guidance_scale = 7.5: controls how strongly the generation is influenced by the prompt.
- num_inference_steps = 300: ensures high-quality and detailed inpainted results.

In [ ]:
def run_inpainting(image, overlay, prompt, out_path):
    result = pipe(
        prompt=prompt,
        image=image,
        mask_image=overlay,
        guidance_scale=7.5,
        num_inference_steps=300,
        height=1024,
        width=1024
    ).images[0]
    result.save(out_path)
    return result

This section loops over all images in the images_to_modify/ directory and applies inpainting-based augmentation using different mask configurations.
- For each image, multiple masks are generated by varying the width (20–30-40%) and side (left or right).
- The fine-tuned model fills the masked region with new crop instances based on the prompt ("sks crop").
- Each output is saved with a suffix indicating the mask parameters, e.g., image_left_30.png.

The function *visualize_result(...)* displays:
- The original image.
- The image with the masked area highlighted in red.
- The inpainted result, showing newly generated crops.

In [ ]:
def visualize_result(original, overlay, inpainted, title):
    red_overlay = np.array(original).copy()
    overlay_np = np.array(overlay.resize(original.size))
    red_overlay[overlay_np > 0] = [255, 0, 0]

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(original)
    axes[0].set_title("Original Image")
    axes[1].imshow(red_overlay)
    axes[1].set_title("overlay Overlay")
    axes[2].imshow(inpainted)
    axes[2].set_title(f"Inpainting: {title}")
    for ax in axes:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

fractions = [0.2, 0.3, 0.4]
directions = ['left', 'right']
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_name in image_files:
    img_path = os.path.join(image_dir, img_name)
    image = Image.open(img_path).convert("RGB")
    base_name = os.path.splitext(img_name)[0]

    for direction in directions:
        for frac in fractions:
            overlay = generate_overlay(image, width_fraction=frac, direction=direction)
            suffix = f"{direction}_{int(frac * 100)}"
            out_name = f"{base_name}_{suffix}.png"
            out_path = os.path.join(output_dir, out_name)

            print(f"Processing {img_name} | {suffix}")
            result = run_inpainting(image, overlay, prompt, out_path)
            visualize_result(image, overlay, result, suffix)